**TELCO CUSTOMER CHURN PREDICTION**

THIS NOTEBOOK PERFORMS:
- DATASET EXPLORATION
- FEATURE ENGINEERING
- DATA CLEANING
- PCA, SHAP VISUALIZATIONS

In [ ]:
#Importing the Libraries

import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

#Loading the Dataset

df=pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

df.info()
df.describe()
df['Churn'].value_counts(normalize=True)

# Visualizing the churn distribution

sns.countplot(x='Churn',data=df)
plt.title("Churn Distribution")